<a href="https://colab.research.google.com/github/861728/ReserveService/blob/main/naver_map_phone_scraper_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 지도 전화번호 자동 수집기 v2
구글 계정 로그인만으로 스프레드시트에 접근합니다. (서비스 계정 불필요)

In [23]:
# ✅ STEP 1: 패키지 설치
!pip install selenium gspread -q
!apt-get update -q
!apt-get install -y chromium-browser chromium-chromedriver -q
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/ 2>/dev/null || true

# 설치 확인
import subprocess
print(subprocess.run(['chromium-browser', '--version'], capture_output=True, text=True).stdout)
print(subprocess.run(['chromedriver', '--version'], capture_output=True, text=True).stdout)

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,212 B]
Fetched 3,037 B in 2s (1,830 B/s)
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Readin

In [24]:
# ✅ STEP 2: 구글 계정 인증 (팝업 뜨면 본인 계정으로 로그인)
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
client = gspread.authorize(creds)
print('✅ 인증 완료!')

✅ 인증 완료!


In [25]:
# ✅ STEP 3: 스프레드시트 연결
# 본인 스프레드시트 URL 붙여넣기
SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/1guEAEM8Vz0VfkTojJXneucjm1RJgsf0KjLTrE2tXFNY/edit?gid=0#gid=0'
SHEET_NAME = '시트2'  # 탭 이름 (한글이면 그대로 입력, 예: '시트1')

sheet = client.open_by_url(SPREADSHEET_URL).worksheet(SHEET_NAME)
all_data = sheet.get_all_values()

print(f'✅ 연결 성공! 총 {len(all_data)-1}행 데이터')
print(f'헤더: {all_data[0]}')

✅ 연결 성공! 총 496행 데이터
헤더: ['PLACE_ID', 'PLACE_NAME', 'SHOP_ADDRESS', '시도', '시군구', 'BIZ_TYPE', 'WAIT_REVIEW_COUNT', '웨이팅 티어', '여부', '타겟 여부', '담당자 ', '검색 시도 날짜\n(날짜찍혀있으면 \n시도완료 한 건으로 간주)\n[<CTRL>+<;>] 키로 날짜 기재해주세요~', '기가맹 여부 =  캐치테이블 입점 유무\n1. 아사나 히스토리 검색 후 인입이력 또는 해지 이력 확인\n2. 내부어드민 매장명 검색', '성공 여부', "키맨직책 / 010 번호\n* 못 찾았을 때에는 'L'열에 실패로 표시해주세요\n*직책기재가 어려울 때는 번호만 적어주셔도 괜찮습니다.", '일반매장번호\n(함께 적어주세요!)', '특이사항']


In [26]:
# ✅ STEP 4: 열 설정 확인
# 이미지 기준: B=2열(PLACE_NAME), O=15열(일반매장번호)
B_COL = 2   # PLACE_NAME 열
O_COL = 15  # 전화번호 기입할 열

headers = all_data[0]
print(f'B열: {headers[B_COL-1]}')
print(f'O열: {headers[O_COL-1] if len(headers) >= O_COL else "(비어있음)"}')

B열: PLACE_NAME
O열: 키맨직책 / 010 번호
* 못 찾았을 때에는 'L'열에 실패로 표시해주세요
*직책기재가 어려울 때는 번호만 적어주셔도 괜찮습니다.


In [35]:
# ✅ STEP 5: 셀레니움 드라이버 초기화

# 구글 크롬 직접 설치
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb -q
!google-chrome --version

# 크롬 버전 확인 후 맞는 chromedriver 설치
!google-chrome --version
!pip install webdriver-manager -q

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time, re, urllib.parse

def create_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

driver = create_driver()
print('✅ 셀레니움 드라이버 초기화 완료!')

Reading package lists...
Building dependency tree...
Reading state information...
google-chrome-stable is already the newest version (145.0.7632.116-1).
0 upgraded, 0 newly installed, 0 to remove and 82 not upgraded.
Google Chrome 145.0.7632.116 
Google Chrome 145.0.7632.116 
✅ 셀레니움 드라이버 초기화 완료!


In [41]:
# ✅ STEP 6 완전 교체 - place ID 직접 추출 방식
import re, time, urllib.parse
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_phone_from_naver(place_name, driver):
    try:
        encoded = urllib.parse.quote(place_name)
        driver.get(f'https://map.naver.com/p/search/{encoded}')
        time.sleep(4)

        wait = WebDriverWait(driver, 10)

        # searchIframe 진입
        iframe = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'iframe#searchIframe')))
        driver.switch_to.frame(iframe)
        time.sleep(2)

        # 첫 번째 가게 링크에서 place ID 추출
        place_id = None
        links = driver.find_elements(By.CSS_SELECTOR, 'a')
        for a in links:
            href = a.get_attribute('href') or ''
            # entryIframe src에서 place ID 추출 시도
            match = re.search(r'/place/(\d+)', href)
            if match:
                place_id = match.group(1)
                break

        driver.switch_to.default_content()

        # place ID로 직접 entryIframe URL 접근
        if not place_id:
            # place ID 없으면 pcmap 직접 접근
            driver.get(f'https://pcmap.place.naver.com/place/list?query={encoded}')
            time.sleep(3)
            src = driver.page_source
            match = re.search(r'"id":"(\d{6,})"', src)
            if match:
                place_id = match.group(1)

        if not place_id:
            return ''

        # place 상세 페이지 직접 접근
        driver.get(f'https://pcmap.place.naver.com/restaurant/{place_id}/home')
        time.sleep(3)

        # 전화번호 추출
        src = driver.page_source

        # 1순위: tel: 링크
        tel_match = re.search(r'href="tel:([^"]+)"', src)
        if tel_match:
            return tel_match.group(1).strip()

        # 2순위: 전화번호 패턴
        matches = re.findall(r'0\d{1,2}-\d{3,4}-\d{4}', src)
        if matches:
            return matches[0]

        return ''

    except Exception as e:
        return ''
    finally:
        driver.switch_to.default_content()

print('✅ 함수 교체 완료!')

✅ 함수 교체 완료!


In [42]:
# ✅ STEP 7: 테스트 (1개만 먼저)
test_name = all_data[1][B_COL-1]
print(f'테스트: {test_name}')
phone = get_phone_from_naver(test_name, driver)
print(f'결과: {phone}')

테스트: 만돈
결과: 031-901-9463


In [ ]:
# ✅ STEP 8: 전체 실행

# ▼▼▼ 여기만 수정하세요 ▼▼▼
START_ROW = 2    # 시작 행 번호 (헤더=1행 제외, 데이터는 2행부터)
END_ROW   = 200   # 종료 행 번호 (이 행까지 처리)
DELAY_SEC = 3    # 요청 간격 (초) - 줄이지 마세요
BATCH     = 10   # 몇 개마다 저장할지
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

updated, skipped, failed = 0, 0, 0
batch_updates = []  # (행번호, 전화번호) 임시 저장

# 최신 데이터 다시 읽기
all_data = sheet.get_all_values()
total_range = END_ROW - START_ROW + 1

print(f'처리 범위: {START_ROW}행 ~ {END_ROW}행 (총 {total_range}개)')
print('=' * 50)

for i in range(START_ROW, END_ROW + 1):
    row = all_data[i-1] if i <= len(all_data) else []
    place_name   = row[B_COL-1].strip() if len(row) >= B_COL else ''
    current_phone = row[O_COL-1].strip() if len(row) >= O_COL else ''

    # 이미 있으면 스킵
    if current_phone:
        print(f'[{i}] ⏭  {place_name} : 스킵 (이미 있음)')
        skipped += 1
        continue

    if not place_name:
        print(f'[{i}] ⚠️  가게명 없음 : 스킵')
        continue

    phone = get_phone_from_naver(place_name, driver)

    if phone:
        batch_updates.append((i, phone))
        print(f'[{i}] ✅  {place_name} : 성공 ({phone})')
        updated += 1
    else:
        print(f'[{i}] ❌  {place_name} : 실패')
        failed += 1

    # 10개마다 일괄 저장
    if len(batch_updates) >= BATCH:
        for row_num, tel in batch_updates:
            sheet.update_cell(row_num, O_COL, tel)
        print(f'\n💾 {len(batch_updates)}개 저장 완료\n')
        batch_updates = []

    time.sleep(DELAY_SEC)

# 남은 것 저장
if batch_updates:
    for row_num, tel in batch_updates:
        sheet.update_cell(row_num, O_COL, tel)
    print(f'\n💾 {len(batch_updates)}개 저장 완료\n')

print('=' * 50)
print(f'검색 완료.  ✅ 성공: {updated}  ❌ 실패: {failed}  ⏭ 스킵: {skipped}')

처리 범위: 2행 ~ 200행 (총 199개)
[2] ✅  만돈 : 성공 (031-901-9463)
[3] ✅  일산칼국수본점 : 성공 (031-903-2208)
[4] ✅  1982성북동왕돈까스 : 성공 (07-1335-5614)
[5] ✅  소리소 시골밥상 남양주 : 성공 (07-1362-9620)
[6] ✅  심학산도토리국수 : 성공 (031-941-3628)
[7] ✅  짬뽕관 하남미사점 : 성공 (07-1321-9587)
[8] ✅  초마 스타필드하남점 : 성공 (080-8681-0150)
[9] ✅  미농앤동동국수집 : 성공 (031-576-1588)
[10] ✅  목향원 : 성공 (031-527-2255)
